# 04 장. 시간 정보와 시간대 다루기

## 4.1. `dateutil`로 시간대 정보 생성하기

In [1]:
import datetime

In [2]:
utc = datetime.datetime.utcnow()

In [3]:
utc

datetime.datetime(2022, 4, 11, 22, 46, 0, 241766)

In [4]:
print(utc.tzinfo)

None


In [5]:
now = datetime.datetime.now()

In [6]:
now

datetime.datetime(2022, 4, 12, 7, 46, 0, 269223)

In [7]:
print(now.tzinfo)

None


---

In [8]:
from dateutil import tz

In [9]:
tz_seoul = tz.gettz('Asia/Seoul')

In [10]:
now.replace(tzinfo=tz_seoul)

datetime.datetime(2022, 4, 12, 7, 46, 0, 269223, tzinfo=tzfile('/usr/share/zoneinfo/Asia/Seoul'))

---

In [11]:
from dateutil.zoneinfo import get_zonefile_instance

In [12]:
zones = list(get_zonefile_instance().zones)

In [13]:
sorted(zones)[:10]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau']

In [14]:
len(zones)

595

---

In [15]:
localzone = tz.gettz()
localzone

tzfile('/etc/localtime')

In [16]:
localzone.tzname(datetime.datetime(2022, 4, 10))

'KST'

## 4.2. 시간대 정보가 있는 `datetime` 객체 직렬화 하기

In [17]:
def utcnow():
    return datetime.datetime.now(tz=tz.tzutc())

In [18]:
utc_ = utcnow()

In [19]:
utc_.isoformat()

'2022-04-11T22:46:00.422348+00:00'

---

In [20]:
def tznow():
    return datetime.datetime.now(tz=tz.gettz())

In [21]:
tz_ = tznow()

In [22]:
tz_.isoformat()

'2022-04-12T07:46:00.432393+09:00'

---

In [23]:
from dateutil.parser import isoparse

In [24]:
parsed = isoparse(utc_.isoformat())
parsed

datetime.datetime(2022, 4, 11, 22, 46, 0, 422348, tzinfo=tzutc())

In [25]:
parsed = isoparse(tz_.isoformat())
parsed

datetime.datetime(2022, 4, 12, 7, 46, 0, 432393, tzinfo=tzoffset(None, 32400))

## 4.3. 애매한 시간대 처리하기

* 2017년 10월 29일 밤, 파리의 서머타임이 해제되었습니다. 이 때 오전 2시에 오전 3시로 변경되었습니다.

In [26]:
localtz = tz.gettz('Europe/Paris')

In [27]:
localtz.is_ambiguous(datetime.datetime(2017, 10, 29, 1, 30))

False

In [28]:
localtz.is_ambiguous(datetime.datetime(2017, 10, 29, 2, 30))

True

In [29]:
localtz.is_ambiguous(datetime.datetime(2017, 10, 29, 3, 30))

False

* 10월 29일 2시 30분이라는 시간 정보를 사용할 때, 서머타임 해제 전의 시간인지 그 후의 시간인지 확인할 수 있도록 `replace` 메소드의 `fold` 키워드 인자에 해당 정보를 공급해야 함

In [30]:
confusing = datetime.datetime(2017, 10, 29, 2, 30, tzinfo=localtz)

In [31]:
confusing.replace(fold=0).astimezone(tz.tzutc())

datetime.datetime(2017, 10, 29, 0, 30, tzinfo=tzutc())

In [32]:
confusing.replace(fold=1).astimezone(tz.tzutc())

datetime.datetime(2017, 10, 29, 1, 30, tzinfo=tzutc())